# Preprocesado de Los Datos

En el notebook "01_Exploracion" se realizó un análisis exploratorio de los datos disponibles. En el presente notebook, se abordará el preprocesamiento de dichos datos con el fin de prepararlos para su posterior utilización.

En particular, no se ha modificado el archivo CSV que contiene la información general de los pacientes, ya que no resulta relevante para la tarea que se pretende resolver. El proceso de preprocesamiento se ha centrado en los 53 archivos CSV individuales que contienen información registrada por los sensores CGM (Continuous Glucose Monitoring).

## 0. Montar Google Drive

Este notebook fue creado y ejecutado en la plataforma Google Colab. Para cargar los datos, se montó Google Drive en el entorno del notebook.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. Carga de los Datos

En primer lugar, se cargaron los 53 archivos CSV y se almacenaron en un diccionario. En dicho diccionario, cada una de las 53 claves corresponde a un paciente, y a cada clave se le asocia el DataFrame correspondiente con los datos del mismo.

Tal como se observó en el notebook de análisis exploratorio y se explicó en la memoria, para el presente trabajo únicamente resultan relevantes dos variables: aquella que contiene la marca temporal (timestamp) y aquella que recoge los valores históricos de glucosa registrados en mg/dL.

In [4]:
import os
import pandas as pd

# Definimos la ruta base donde están los archivos CSV
ruta_datos = '/content/drive/My Drive/TFG/Datos'
# Preparamos el diccionario para almacenar los DataFrames de cada paciente
datos_pacientes = {}

for i in range(1, 54):
    nombre_archivo = f'811-{i}.csv'
    ruta_archivo = os.path.join(ruta_datos, nombre_archivo)
    try:
        # Cargamos los datos y eliminamos filas con valores ausentes
        df = pd.read_csv(
            ruta_archivo,
            skiprows=2,
            # COLUMNAS DE INTERES
            usecols=[
                'Historial de glucosa mg/dL',
                'Sello de tiempo del dispositivo'
            ]
        ).dropna().copy()

        # Normalizamos los nombres de las columnas para el análisis
        df.rename(columns={
            'Historial de glucosa mg/dL': 'Historial_Glucosa',
            'Sello de tiempo del dispositivo': 'Timestamp'
        }, inplace=True)

        # Almacenamos el DataFrame correspondiente al paciente i
        datos_pacientes[f'paciente{i}'] = df

    except Exception as error:
        print(f"Error al procesar {nombre_archivo}: {error}")


In [5]:
x = 1
datos_pacientes[f'paciente{x}']

,Timestamp,Historial_Glucosa
34,23/07/2022 1:07,415.0
35,23/07/2022 1:22,417.0
36,23/07/2022 1:37,439.0
37,23/07/2022 1:52,424.0
38,23/07/2022 2:07,408.0
...,...,...
24948,29/02/2024 8:32,297.0
24949,29/02/2024 8:47,269.0
24950,29/02/2024 9:02,252.0
24951,29/02/2024 9:17,300.0


## 2. Preprocesado de Los Datos

In [10]:
from sklearn.preprocessing import MinMaxScaler

# Concatenamos todos los valores de glucosa y creamos un DataFrame con nombre de columna
valores_glucosa = pd.concat(
    [df['Historial_Glucosa'] for df in datos_pacientes.values()]
)

valores_glucosa_df = valores_glucosa.to_frame(name='Historial_Glucosa')

# Ajustamos el scaler usando el DataFrame
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(valores_glucosa_df)

# Preprocesamos cada paciente usando el scaler ya ajustado
datos_preprocesados = {}
for paciente_id, df in datos_pacientes.items():
    df = df.copy()

    # Convertimos el sello de tiempo y ordenamos las medicione
    df['Timestamp'] = pd.to_datetime(
        df['Timestamp'], dayfirst=True, errors='coerce')

    df = df.sort_values('Timestamp').set_index('Timestamp')

    # Remuestreamos cada 15 minutos y rellenamos huecos por interpolación lineal
    df = (
        df.resample('15min')
          .mean()
          .interpolate(method='linear')
          .reset_index()
    )

    # Normalizamos los valores de glucosa entre 0 y 1
    df['Glucosa_Normalizada'] = scaler.transform(
        df[['Historial_Glucosa']]
    )

    datos_preprocesados[paciente_id] = df

In [9]:
x = 1
datos_preprocesados[f'paciente{x}']

,Timestamp,Historial_Glucosa,Glucosa_Normalizada
0,2022-07-23 01:00:00,415.0,0.815217
1,2022-07-23 01:15:00,417.0,0.819565
2,2022-07-23 01:30:00,439.0,0.867391
3,2022-07-23 01:45:00,424.0,0.834783
4,2022-07-23 02:00:00,408.0,0.800000
...,...,...,...
56286,2024-02-29 08:30:00,297.0,0.558696
56287,2024-02-29 08:45:00,269.0,0.497826
56288,2024-02-29 09:00:00,252.0,0.460870
56289,2024-02-29 09:15:00,300.0,0.565217


**NOTA:** Cabe destacar que, durante el proceso de preprocesamiento, se añadió una columna con los valores de glucosa normalizados utilizando la técnica MinMaxScaler. No obstante, en este trabajo no se emplean dichos valores normalizados, ya que los modelos deben ser capaces de aprender a predecir en cualquier rango de glucosa.

De hecho, aunque se realizaron algunas pruebas y experimentos preliminares utilizando la glucosa normalizada, estos no tuvieron continuidad. Por tanto, dicha columna no es esencial y podría eliminarse sin afectar al desarrollo del trabajo.

Como podemos observar, la mayoría de los DataFrames que mostramos en pantalla contienen una gran cantidad de valores NA y columnas que parecen carecer de información relevante. En la siguiente sección, realizaremos un análisis exploratorio más detallado a nivel individual.

In [11]:
import numpy as np

"""
Creación de las series temporales para entrenar modelos. En este caso para un
horizonte de 15 minutos. (h=1)
"""

# Definimos el mapeo de cada paciente a un índice numérico
claves = list(datos_preprocesados.keys())
mapeo_paciente_indice = {paciente: idx for idx, paciente in enumerate(claves)}

# Inicializamos las listas para secuencias, etiquetas y paciente índice
secuencias, etiquetas, paciente_indices = [], [], []

# Generamos ventanas deslizantes de glucosa por paciente
for paciente, df in datos_preprocesados.items():
    valores_glucosa = df['Historial_Glucosa'].values

    # Para cada bloque de 10 registros, guardamos la secuencia y su etiqueta
    for inicio in range(0, len(valores_glucosa) - 10, 10):
        ventana = valores_glucosa[inicio: inicio + 10]
        objetivo = valores_glucosa[inicio + 10]

        secuencias.append(ventana)
        etiquetas.append(objetivo)
        paciente_indices.append(mapeo_paciente_indice[paciente])

# Convertimos a arrays NumPy con la forma requerida para modelos de series temporales
X_secuencias = np.array(secuencias).reshape(-1, 10, 1)
X_paciente = np.array(paciente_indices)
y = np.array(etiquetas)

In [17]:
# Mostramos un ejemplo de secuencia y etiqueta para el paciente 1

# Seleccionamos la primera ventana de 10 valores y su etiqueta
ventana_ejemplo = X_secuencias[0].flatten()
etiqueta_ejemplo = y[0]

# Mostramos la ventana de glucosa (10 valores consecutivos)
print("Ventana de glucosa (10 lecturas consecutivas):")
print(ventana_ejemplo)

# Mostramos la etiqueta (valor de glucosa inmediato siguiente)
print("\nEtiqueta asociada (siguiente lectura de glucosa):")
print(etiqueta_ejemplo)

# Mostramos los primeros 11 registros originales para verificar correspondencia
print("\nPrimeros 11 registros de paciente1:")
display(datos_preprocesados['paciente1'].head(11))





Ventana de glucosa (10 lecturas consecutivas):
[415. 417. 439. 424. 408. 409. 397. 376. 365. 354.]

Etiqueta asociada (siguiente lectura de glucosa):
368.0

Primeros 11 registros de paciente1:


,Timestamp,Historial_Glucosa,Glucosa_Normalizada
0,2022-07-23 01:00:00,415.0,0.815217
1,2022-07-23 01:15:00,417.0,0.819565
2,2022-07-23 01:30:00,439.0,0.867391
3,2022-07-23 01:45:00,424.0,0.834783
4,2022-07-23 02:00:00,408.0,0.800000
5,2022-07-23 02:15:00,409.0,0.802174
6,2022-07-23 02:30:00,397.0,0.776087
7,2022-07-23 02:45:00,376.0,0.730435
8,2022-07-23 03:00:00,365.0,0.706522
9,2022-07-23 03:15:00,354.0,0.682609


In [20]:
# Obtenemos las dimensiones de nuestro array de secuencias
n_muestras, longitud_ventana, n_caracteristicas = X_secuencias.shape

# Mostramos la información de forma clara
print(f"Número de muestras: {n_muestras}")
print(f"Longitud de cada secuencia (timesteps): {longitud_ventana}")
print(f"Número de características por timestep: {n_caracteristicas}")


Número de muestras: 320142
Longitud de cada secuencia (timesteps): 10
Número de características por timestep: 1
